In [1]:
import pandas_datareader.data as web
import pandas as pd
from datetime import datetime, date, timedelta
from dateutil.relativedelta import relativedelta
from quantAPP.config import Config
from sqlalchemy import create_engine
import pandas_datareader.data as web
from typing import Dict
import yfinance as yf
import pandas as pd


config = Config()
path = r"C:\Users\wwwes\Documents\wwweslei-quantApp\quantAPP\ext\profit\company_files"
engine = create_engine(config.SQLALCHEMY_DATABASE_URI,
                       echo=config.SQLALCHEMY_ECHO)

def create_dictionary(df):
    df = df.groupby(pd.Grouper(freq='M')).last()[-13:]['Close']
    data = {month: float(close/df.head(1)-1)
            for (month, close) in zip(df.index.strftime('%m/%y'), df)}
    return data
def wdo(is_update: bool = False) -> pd.DataFrame:
    if is_update:
        df = web.DataReader("usd/brl", "av-forex-daily",
                            api_key=config.API_KEY)
        df.index = pd.to_datetime(df.index)
        df.columns = ["Open", "High", "Low", "Close"]
        df.index.names = ["Date"]
        df.to_sql("WDO", con=engine, if_exists="replace")
        return df
    df = pd.read_sql("SELECT * FROM WDO", con=engine, index_col="Date")
    df.index = pd.to_datetime(df.index)
    return df
def stock(ticket: str, is_update: bool = False) -> pd.DataFrame:
    if is_update:
        df = yf.download(ticket)
        df.to_sql(ticket, con=engine, if_exists="replace")
        return df
    df = pd.read_sql(f"SELECT * FROM {ticket}", con=engine, index_col="Date")
    df.index = pd.to_datetime(df.index)
    return df
df = stock('PETR4')
mes12 = df.groupby(pd.Grouper(freq='M')).last()[-13:-12]['Close']
annual = df.groupby(pd.Grouper(freq='M')).last()[-(date.today().month+1):]['Close'].head(1)
week = df.groupby(pd.Grouper(freq='w')).last()[-2:-1]['Close']

In [22]:
df[df.index < datetime.today() - relativedelta(years=1)].tail(1)

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-07-09,22.780001,22.809999,22.059999,22.139999,21.418694,57151500
